In [1]:
#Results were produced in stints. This is the number of the last stint.
run = 19

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from scipy import stats
from scipy import optimize
import joblib

#folder for saving results
filepath = ".../Resultate_final/PTF/IS/PTF_VaR_ES_IS_sim_saved/"

KeyboardInterrupt: 

In [ ]:
#Market and option parameters as in section 4.2 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
s_0 = 100
r = 0.01
corr= 0.3
tau = 1/52
T = 1/3
K = 100

mu = np.empty(20)
sigma = np.empty(20)
for i in range(0,10):
    mu[i] = mu[i+10] = (3+(i+1)/2)/100
    sigma[i] = sigma[i+10] = (15+(i+1))/100

cov_mat = np.empty((20,20))
for i in range(0,20):
    for j in range(0,20):
        if i != j:
            cov_mat[i,j] = corr
        else:
            cov_mat[i,j] = 1

C = np.linalg.cholesky(cov_mat)

#Confidence levels and parameters for Value-at-Risk and Expected Shortfall
alpha_VaR = 0.995
alpha_ES = 0.99

In [ ]:
#Sizes for training set, validation set, test set, and set size for Monte Carlo estimation of the risk measures
M_1 = 1500000
M_2 = 500000
M_3 = 500000
#ignore N or N_2 in the following. Was kept just in case, but not used.
N_2 = 1
M_MC = 500000
#size of the set of data points used to calculate an IS density
M_IS = 1000000
#quantile for which the IS density will be computed
alpha_IS = 0.995

In [ ]:
#Function for calculating simulated values of S_tau and simulated payoffs P_T from simulations of multivariate standard normal random variables
def data_gen(Z,V):
    #correlating the independent components of Z
    Z = np.transpose(np.matmul(C,np.transpose(Z)))
    
    #simulate S_tau under P
    S_tau = np.empty((len(Z), 20))
    for j in range(0,20):
        S_tau[:,j] = s_0 * np.exp( (mu[j]-0.5*sigma[j]**2)*tau + np.sqrt(tau)*sigma[j]*Z[:,j] )

    #simulate S_T given S_tau under Q
    S_T = np.empty((len(Z),20))
    for j in range(0,20):
        S_T[:,j] = S_tau[:,j] * np.exp( (r-0.5*sigma[j]**2)*(T-tau) + np.sqrt(T-tau)*sigma[j]*V[:,j] )

    #compute discounted option payoffs
    P_T_pre =np.empty((len(S_T), 20))
    for j in range(0,10):
        P_T_pre[:,j] = np.exp(-r*(T-tau)) * np.maximum(S_T[:,j]-K,0)
    for j in range(10,20):
        P_T_pre[:,j] = np.exp(-r*(T-tau)) * np.maximum(K-S_T[:,j],0)
    P_T = np.sum(P_T_pre, axis=1)
    return S_tau,P_T

#the function DT(Z,\theta)
def data_trans_IS(Z,IS):
    res = np.empty((len(Z),20))
    for j in range(20):
        res[:,j] = Z[:,j]*np.sqrt(IS[20+j]) + IS[j]
    return res

#The density function of Z
def f(y):
    return stats.multivariate_normal.pdf(y, mean=np.full(20,0), cov=np.identity(20))

#The density function of Z_\theta (note that x is interpreted as theta, needed for the least-squares solver to work properly)
def f_theta(y, x):
    return stats.multivariate_normal.pdf(y, mean=x[0:20], cov=np.diag(x[20:40]), allow_singular=True)

#This function describes the approximation of the expression inside the sum of m_2(theta)
def g_q_alpha_hat_reweighted(x,L,q_alpha_hat):
    return np.sqrt(f(y=L[:,0:20])/f_theta(y=L[:,0:20],x=x))*(L[:,-1]>q_alpha_hat)

#bounds for the IS density parameters (for the parameters corresponding to the mean no bounds are necessary, the standard deviation parameters however needs to be non-negative)
bnds_lower = np.empty((40))
bnds_upper = np.empty((40))
for j in range(20):
    bnds_lower[j] = -np.inf
    bnds_upper[j] = np.inf
    bnds_lower[20+j] = 0
    bnds_upper[20+j] = np.inf
    
bnds = (bnds_lower, bnds_upper)

In [ ]:
for j in range(84):
    #Generating realisations of multivariate standard normal random variables (uncorrelated for Z such that calculation of \theta^* is possible and correlated for V)
    Z_IS = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_IS)
    V_IS = np.random.multivariate_normal(mean=np.full(20,0), cov=cov_mat, size=M_IS)

    #Calculate the risk factor S_tau and the corresponding simulated payoffs P_T
    S_tau_IS, P_T_IS = data_gen(Z=Z_IS, V=V_IS)

    #define and compile neural network model, setup as in section 4.2 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
    bi_IS = np.log( np.sum(P_T_IS)/len(P_T_IS))
    model_IS = tf.keras.models.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(20,)),
        tf.keras.layers.Dense(20, activation='tanh'),
        tf.keras.layers.Dense(20, activation='tanh'),
        tf.keras.layers.Dense(1, activation='exponential', bias_initializer=tf.keras.initializers.Constant(value=bi_IS))])
    model_IS.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model_IS.fit(x=S_tau_IS, y=P_T_IS, epochs=100, batch_size=10000, verbose=0)

    #Calculate realisations of L_hat from the training data set using the trained neural network
    L_hat_IS = np.column_stack((Z_IS, model_IS.predict(S_tau_IS)[:,0]))
    L_hat_IS_sort = L_hat_IS[L_hat_IS[:,-1].argsort()[::-1]]

    #Calculating the corresponding estimator for Value-at-Risk in order to approximate g
    q_alpha_IS_hat = L_hat_IS_sort[int(M_IS*(1-alpha_IS)-1), -1]
    print('q_alpha_IS_hat_NN:',q_alpha_IS_hat)

    #Calculating the (hopefully) approximately \theta^*_{NN} by minimising m_2 using the approximated g
    IS_NN = optimize.least_squares(g_q_alpha_hat_reweighted, x0=np.concatenate((np.full(20,0),np.full(20,1))), args=(L_hat_IS, q_alpha_IS_hat), bounds=bnds).x

    #define and train a random forest according to the optimal parameters from tuning
    rfr_IS = RandomForestRegressor(n_estimators=160, criterion='squared_error', max_features=8, min_samples_leaf=70, bootstrap=True, verbose=0, n_jobs=-1)
    rfr_IS.fit(X=S_tau_IS, y=P_T_IS)

    #Calculate realisations of L_hat from the training data set using the trained random forest
    L_hat_IS = np.column_stack((Z_IS, rfr_IS.predict(S_tau_IS)))
    L_hat_IS_sort = L_hat_IS[L_hat_IS[:,-1].argsort()[::-1]]

    #Calculating the corresponding estimator for Value-at-Risk in order to approximate g
    q_alpha_IS_hat = L_hat_IS_sort[int(M_IS*(1-alpha_IS)-1), -1]
    print('q_alpha_IS_hat_RF:',q_alpha_IS_hat)

    #Calculating the (hopefully) approximately optimal \theta^*_{RF} by minimising m_2 using the approximated g
    IS_RF = optimize.least_squares(g_q_alpha_hat_reweighted, x0=np.concatenate((np.full(20,0),np.full(20,1))), args=(L_hat_IS, q_alpha_IS_hat),bounds=bnds).x

    #print IS density parameters for checking
    print('IS_NN:',IS_NN)
    print('IS_RF:',IS_RF)
    
    #Generating simulations for multivariate standard normal random variables (uncorrelated for Z, correlated for V) for training set, validation set, test set, set for Monte Carlo estimation of risk measures
    Z_train = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_1)
    V_train = np.random.multivariate_normal(mean=np.full(20,0), cov=cov_mat, size=M_1)
    Z_val = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_2)
    V_val = np.random.multivariate_normal(mean=np.full(20,0), cov=cov_mat, size=M_2)
    Z_test = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_3)
    V_test = np.random.multivariate_normal(mean=np.full(20,0), cov=cov_mat, size=M_3)
    Z_MC = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_MC)
    V_MC = np.random.multivariate_normal(mean=np.full(20,0), cov=cov_mat, size=M_MC)

    #calculating DT(Z,\theta^*_{NN})
    Z_train_NN = data_trans_IS(Z_train,IS_NN)
    Z_val_NN = data_trans_IS(Z_val,IS_NN)
    Z_test_NN = data_trans_IS(Z_test,IS_NN)
    Z_MC_NN = data_trans_IS(Z_MC,IS_NN)
    #calculating the risk factors under the IS distribution and corresponding option prices
    S_tau_train_NN, P_T_train_NN = data_gen(Z=Z_train_NN, V=V_train)
    S_tau_val_NN, P_T_val_NN = data_gen(Z=Z_val_NN, V=V_val)
    S_tau_test_NN, P_T_test_NN = data_gen(Z=Z_test_NN, V=V_test)
    S_tau_MC_NN, P_T_MC_NN = data_gen(Z=Z_MC_NN, V=V_MC)

    #calculating DT(Z,\theta^*_{RF})
    Z_train_RF = data_trans_IS(Z_train,IS_RF)
    Z_val_RF = data_trans_IS(Z_val,IS_RF)
    Z_test_RF = data_trans_IS(Z_test,IS_RF)
    Z_MC_RF = data_trans_IS(Z_MC,IS_RF)
    #calculating the risk factors under the IS distribution and corresponding option prices
    S_tau_train_RF, P_T_train_RF = data_gen(Z=Z_train_RF, V=V_train)
    S_tau_val_RF, P_T_val_RF = data_gen(Z=Z_val_RF, V=V_val)
    S_tau_test_RF, P_T_test_RF = data_gen(Z=Z_test_RF, V=V_test)
    S_tau_MC_RF, P_T_MC_RF = data_gen(Z=Z_MC_RF, V=V_MC)
    
    #calculating parameters for the sets B_1 and B_2
    s_20_1 = s_0 * np.exp((mu[0:3]-0.5*sigma[0:3]**2)*tau + sigma[0:3]*np.sqrt(tau)*stats.norm.ppf(0.2, loc=0, scale=1))
    s_80_1 = s_0 * np.exp((mu[9:12]-0.5*sigma[9:12]**2)*tau + sigma[9:12]*np.sqrt(tau)*stats.norm.ppf(0.8, loc=0, scale=1))
    s_20_2 = s_0 * np.exp((mu[9:12]-0.5*sigma[9:12]**2)*tau + sigma[9:12]*np.sqrt(tau)*stats.norm.ppf(0.2, loc=0, scale=1))
    s_80_2 = s_0 * np.exp((mu[0:3]-0.5*sigma[0:3]**2)*tau + sigma[0:3]*np.sqrt(tau)*stats.norm.ppf(0.8, loc=0, scale=1))

    #calculate the indices of the set B_1 and B_2 for the test set created with the IS density calculated by the neural network
    B_1_NN = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (np.apply_along_axis(np.all, axis=1, arr=S_tau_test_NN[:,0:3] > s_20_1), np.apply_along_axis(np.all, axis=1, arr=S_tau_test_NN[:,9:12] < s_80_1)) ) )
    B_2_NN = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (np.apply_along_axis(np.all, axis=1, arr=S_tau_test_NN[:,0:3] < s_80_2), np.apply_along_axis(np.all, axis=1, arr=S_tau_test_NN[:,9:12] > s_20_2)) ) )

    #calculate the indices of the set B_1 and B_2 for the test set created with the IS density calculated by the random forest
    B_1_RF = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (np.apply_along_axis(np.all, axis=1, arr=S_tau_test_RF[:,0:3] > s_20_1), np.apply_along_axis(np.all, axis=1, arr=S_tau_test_RF[:,9:12] < s_80_1)) ) )
    B_2_RF = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (np.apply_along_axis(np.all, axis=1, arr=S_tau_test_RF[:,0:3] < s_80_2), np.apply_along_axis(np.all, axis=1, arr=S_tau_test_RF[:,9:12] > s_20_2)) ) )
    
    #define and compile neural network model, setup as in section 4.2 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
    bi = np.log( np.sum(P_T_train_NN)/len(P_T_train_NN))
    model = tf.keras.models.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(20,)),
        tf.keras.layers.Dense(20, activation='tanh'),
        tf.keras.layers.Dense(20, activation='tanh'),
        tf.keras.layers.Dense(1, activation='exponential', bias_initializer=tf.keras.initializers.Constant(value=bi))])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    #training the neural network
    hist = model.fit(x=S_tau_train_NN, y=P_T_train_NN, epochs=100, batch_size=10000, validation_data=(S_tau_val_NN,P_T_val_NN), verbose=0)
    
    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 for the neural network
    P_T_pred_NN = model.predict(S_tau_test_NN)[:,0]
    mse_train_NN = hist.history['mse'][-1]
    mse_val_NN = hist.history['val_mse'][-1]
    mc_tmp = P_T_pred_NN - P_T_test_NN
    metric_a_NN = np.sum(mc_tmp)/len(P_T_test_NN)
    metric_b_NN = np.sum((mc_tmp)*P_T_pred_NN)/len(P_T_test_NN)
    metric_c_B_1_NN = np.sum(mc_tmp[B_1_NN])/len(P_T_test_NN)
    metric_c_B_2_NN = np.sum(mc_tmp[B_2_NN])/len(P_T_test_NN)

    #computation of option price depending on the risk factor S_tau according to the models, i.e. computation of L_hat_i's
    L_hat_NN = model.predict(S_tau_MC_NN)[:,0]
    L_hat_c_NN = np.column_stack((Z_MC_NN, L_hat_NN))
    
    #calculation of the IS estimator for Value-at-Risk and Expected Shortfall
    L_hat_c_sort_NN = L_hat_c_NN[L_hat_c_NN[:,-1].argsort()[::-1]]
    w = f(L_hat_c_sort_NN[:,0:20])/(M_MC*f_theta(x=IS_NN, y=L_hat_c_sort_NN[:,0:20]))

    j_VaR = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_VaR) and j_VaR<M_MC):
        w_sum_tmp += w[j_VaR]
        j_VaR += 1
    VaR_hat_NN = L_hat_c_sort_NN[j_VaR,-1]
    print('VaR_hat_NN:',VaR_hat_NN)
    
    j_ES = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_ES) and j_ES<M_MC):
        w_sum_tmp += w[j_ES]
        j_ES += 1
    ES_hat_NN = (1/(1-alpha_ES)) * np.sum(w[0:j_ES-1]*L_hat_c_sort_NN[0:j_ES-1,-1]) + ( 1 - (1 / (1-alpha_ES)) * np.sum(w[0:j_ES-1]) )*L_hat_c_sort_NN[j_ES,-1]
    print('ES_hat_NN:',ES_hat_NN)
    
    
    #perform a grid search in order to find the (approximately) best hyperparameter min_samples_leaf
    #values that will be checked
    max_features_list = [8]
    min_samples_leaf_list = [30,40,50]
    opt_param = np.full(2,0)
    opt_score = np.inf

    for max_features in max_features_list:
        for min_samples_leaf in min_samples_leaf_list:
            rfr_tuning = RandomForestRegressor(n_estimators=160, max_features=max_features, min_samples_leaf=min_samples_leaf, bootstrap=True, criterion='squared_error', verbose=0, n_jobs=-1)
            rfr_tuning.fit(X=S_tau_train_RF, y=P_T_train_RF)
            score = mean_squared_error(y_true=P_T_val_RF, y_pred=rfr_tuning.predict(S_tau_val_RF))
            if score < opt_score:
                opt_param_RF = np.array([max_features,min_samples_leaf])
                opt_score = score
    
    #definition and training of random forest regressor
    rfr = RandomForestRegressor(n_estimators=400, criterion='squared_error', max_features=int(opt_param_RF[0]), min_samples_leaf=int(opt_param_RF[1]), bootstrap=True, verbose=0, warm_start=True, n_jobs=-1)
    rfr.fit(X=S_tau_train_RF, y=P_T_train_RF)
    
    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 and training/valdiation MSE for the random forest
    mse_train_RF = mean_squared_error(y_true=P_T_train_RF, y_pred=rfr.predict(S_tau_train_RF))
    mse_val_RF = mean_squared_error(y_true=P_T_val_RF, y_pred=rfr.predict(S_tau_val_RF))
    P_T_pred_RF = rfr.predict(S_tau_test_RF)
    mc_tmp = P_T_pred_RF - P_T_test_RF
    metric_a_RF = np.sum(mc_tmp)/len(P_T_test_RF)
    metric_b_RF = np.sum((mc_tmp)*P_T_pred_RF)/len(P_T_test_RF)
    metric_c_B_1_RF = np.sum(mc_tmp[B_1_RF])/len(P_T_test_RF)
    metric_c_B_2_RF = np.sum(mc_tmp[B_2_RF])/len(P_T_test_RF)

    #computation of option price depending on the risk factor S_tau according to the models, i.e. computation of L_hat_i's
    L_hat_RF = rfr.predict(S_tau_MC_RF)
    L_hat_c_RF = np.column_stack((Z_MC_RF, L_hat_RF))
    
    #calculation of the IS estimators for Value-at-Risk and Expected Shortfall
    L_hat_c_sort_RF = L_hat_c_RF[L_hat_c_RF[:,-1].argsort()[::-1]]
    w = f(L_hat_c_sort_RF[:,0:20])/(M_MC*f_theta(x=IS_RF, y=L_hat_c_sort_RF[:,0:20]))

    j_VaR = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_VaR) and j_VaR<M_MC):
        w_sum_tmp += w[j_VaR]
        j_VaR += 1
    VaR_hat_RF = L_hat_c_sort_RF[j_VaR,-1]
    print('VaR_hat_RF:',VaR_hat_RF)
    
    j_ES = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_ES) and j_ES<M_MC):
        w_sum_tmp += w[j_ES]
        j_ES += 1
    ES_hat_RF = (1/(1-alpha_ES)) * np.sum(w[0:j_ES-1]*L_hat_c_sort_RF[0:j_ES-1,-1]) + ( 1 - (1 / (1-alpha_ES)) * np.sum(w[0:j_ES-1]) )*L_hat_c_sort_RF[j_ES,-1]
    print('ES_hat_RF:',ES_hat_RF)
    
    #save results for further evaluation
    output = np.array([[mse_train_NN,mse_val_NN,metric_a_NN,metric_b_NN,metric_c_B_1_NN,metric_c_B_2_NN,VaR_hat_NN,ES_hat_NN],
                      [mse_train_RF,mse_val_RF,metric_a_RF,metric_b_RF,metric_c_B_1_RF,metric_c_B_2_RF,VaR_hat_RF,ES_hat_RF]])

    joblib.dump(output,filepath+'output_'+str(run)+'_'+str(j)+'.joblib')
    #since IS_NN and IS_RF are so high-dimensional we save them separately in order to avoid confusion in the later evaluation of results
    joblib.dump(IS_NN, '.../Resultate_final/PTF/IS/PTF_VaR_ES_IS_sim_NN_densities_saved/'+'output_'+str(run)+'_'+str(j)+'.joblib')
    joblib.dump(IS_RF, '.../Resultate_final/PTF/IS/PTF_VaR_ES_IS_sim_RF_densities_saved/'+'output_'+str(run)+'_'+str(j)+'.joblib')
    #prints just for checking while the notebook is running
    print(j)